# Dataframe from JSON


In [19]:
import pyspark
import pyspark.sql.functions as F
import pandas as pd
import re
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, MapType, ArrayType
from pyspark.sql.functions import udf


In [20]:
# Getting session from Spark
spark = SparkSession.builder \
.appName('test').master('local[*]') \
.config("spark.cores.max", "2") \
.config("spark.executor.memory", "2g") \
.config("spark.sql.execution.arrow.pyspark.enabled", "true") \
.config("spark.shuffle.service.enabled", "false") \
.config("spark.dynamicAllocation.enabled", "true") \
.getOrCreate()




### A proper JSON format in file dataframes/data/test.js

```json
[
    {
        "name": "Andre",
        "id": 1,
        "doc_list":[{"docid":"DOC001", "name":"bla001.txt"}, {"docid":"DOC002", "name":"bla002.txt"}],
    },

    {
        "name": "Noé",
        "id": 2,
        "doc_list":[{"docid":"DOC003", "name":"bla003.txt"}, {"docid":"DOC004", "name":"bla004.txt"}],
    }
]


```



### Example problem

Based on JSON structure in the file 'test.js'(shown above), extract the doc file names associated to the people's names as following: 

```text
+-----+----------------------+
|name |doc_names             |
+-----+----------------------+
|Andre|bla001.txt, bla002.txt|
|Noé  |bla003.txt, bla004.txt|
+-----+----------------------+
```

Note that, 'docid' is not desirable.



In [4]:
# Defining schema
schema = StructType([
    StructField("name", StringType(), True),
    StructField("doc_list", ArrayType(MapType(StringType(),StringType(),True),True), True),
])
# Reading JSON file using Dataframe API setting 'multiline' option as true
sdf = spark.read.option("multiline", "true").json('data/test.json', schema=schema)
sdf.show(truncate=False)

+-----+------------------------------------------------------------------------------+
|name |doc_list                                                                      |
+-----+------------------------------------------------------------------------------+
|Andre|[{docid -> DOC001, name -> bla001.txt}, {docid -> DOC002, name -> bla002.txt}]|
|Noé  |[{docid -> DOC003, name -> bla003.txt}, {docid -> DOC004, name -> bla004.txt}]|
+-----+------------------------------------------------------------------------------+



### Transformations steps

1. Transform the list in 'doc_list' column into various rows using the `explode` pyspark function;
2. For each of those rows, extract 'name' from the data structure using `getItem` function and drop the original column 'doc_ex';
3. Transform doc names into a list againd group rows by 'name' column and using the `collect_list` as the aggregate function;
4. Transform the list into a string separated by ',' using `concat_ws`


In [5]:
# Step 1
adf = sdf.withColumn("doc_ex", F.explode("doc_list"))
print("Step 1")
adf.show(truncate=False)
# Step 2 - Extracting the value of interest. In this case, the names of documents.
adf = adf.withColumn("doc_name", adf.doc_ex.getItem("name")).drop("doc_ex")
print("Step 2")
adf.show(truncate=False)
# Step 3 - Time to revert the 'explode' effect. For this, let's group rows by name and use 'collect_list' as aggregate function
ndf = adf.groupBy("name").agg(F.collect_list("doc_name").alias('doc_list'))
print("Step 3")
ndf.show(truncate=False)
# Step 4 - Transforming this list into a string list separated by ',' character.
ndf = ndf.withColumn("doc_names", F.concat_ws(",", "doc_list")).drop("doc_list")
print("Step 4")
ndf.show(truncate=False)

Step 1
+-----+------------------------------------------------------------------------------+-------------------------------------+
|name |doc_list                                                                      |doc_ex                               |
+-----+------------------------------------------------------------------------------+-------------------------------------+
|Andre|[{docid -> DOC001, name -> bla001.txt}, {docid -> DOC002, name -> bla002.txt}]|{docid -> DOC001, name -> bla001.txt}|
|Andre|[{docid -> DOC001, name -> bla001.txt}, {docid -> DOC002, name -> bla002.txt}]|{docid -> DOC002, name -> bla002.txt}|
|Noé  |[{docid -> DOC003, name -> bla003.txt}, {docid -> DOC004, name -> bla004.txt}]|{docid -> DOC003, name -> bla003.txt}|
|Noé  |[{docid -> DOC003, name -> bla003.txt}, {docid -> DOC004, name -> bla004.txt}]|{docid -> DOC004, name -> bla004.txt}|
+-----+------------------------------------------------------------------------------+--------------------------------

### UDF(Not recommended)
User Defined Functions(UDF) can be a way to parse information from a column. In this case, the docs inside the JSON file is available in a list of objects which is parsed by pySpark and convenient converted into Python data structure objects which looks like more simpler to deal with. However, in this particularly scenario is not recommended because is possible to use spark functions which supports this operation offering a better optimization than UDF. Besides, in Python performance is not particularly good specially if you have neasted loops.

In [6]:
@udf
def extract_doc(data_list: list) -> str:
    n = list()
    for li in data_list:
        n += [v for k,v in li.items() if k == 'name']

    return ','.join(n)


In [7]:
# Running the UDF called 'extract_doc_udf' and storing into a new column called 'udf_res'
dfu = sdf.withColumn('doc_names', extract_doc(F.col('doc_list'))).select('name','doc_names')
# Showing the result
dfu.show(truncate=False)

+-----+---------------------+
|name |doc_names            |
+-----+---------------------+
|Andre|bla001.txt,bla002.txt|
|Noé  |bla003.txt,bla004.txt|
+-----+---------------------+



### "UDF" using RDD(less recommended)
This is the old ways to handle UDFs. The reasons to not to do it is the same as the previous ways. And here is worse because you dealing directly with RDD and you will not have any optimization for doing that. If you don't know how to optimize RDD operations by yourself, don't use it.

In [8]:
def extract_doc_rdd(row):
    d = row.asDict()
    n = list()
    if 'doc_list' in d:
        for li in d['doc_list']:
            n += [v for k,v in li.items() if k == 'name']

        d['doc_names'] = ','.join(n)

    return Row(**d)
    


In [9]:
# Executing 'extract_doc_rdd' using map method from rdd object
rdd = sdf.rdd.map(extract_doc_rdd)
# Converting into a dataframe object
edf = rdd.toDF().select('name','doc_names')
# Showing the result
edf.show(truncate=False)

+-----+---------------------+
|name |doc_names            |
+-----+---------------------+
|Andre|bla001.txt,bla002.txt|
|Noé  |bla003.txt,bla004.txt|
+-----+---------------------+



### Reading complex JSON using dynamic schema

For the cases where JSON contains varying all the time. Example on dataframe/data/json-varying.csv:

```json
id,json_string
1,'{"name": "John Doe", "age": 30}'
2,'{"city": "New York", "country": "USA", "zipcode": "10001"}'
3,'{"product": "Laptop", "brand": "Dell", "specs": {"RAM": "16GB", "Storage": "512GB SSD"}}'

```

Tipically this kind of data came from a column



In [16]:
# Or from file.
df = spark.read.csv('data/json-varying.csv', header=True, quote="'")

# If you're reading from a file, remove the quote char from string. Otherwise, the parser will not be able to return a object
df = df.withColumn('json_string', F.regexp_replace('json_string', r"\'", ""))
df.show(truncate=False)

+---+----------------------------------------------------------------------------------------+
|id |json_string                                                                             |
+---+----------------------------------------------------------------------------------------+
|1  |{"name": "John Doe", "age": 30}                                                         |
|2  |{"city": "New York", "country": "USA", "zipcode": "10001"}                              |
|3  |{"product": "Laptop", "brand": "Dell", "specs": {"RAM": "16GB", "Storage": "512GB SSD"}}|
+---+----------------------------------------------------------------------------------------+



In [14]:
dynamic_schema = spark.read.json(df.rdd.map(lambda row: row['json_string'])).schema
jdf = df.withColumn("parsed", F.from_json('json_string', dynamic_schema))
jdf.show(truncate=False)

+---+----------------------------------------------------------------------------------------+---------------------------------------------------------------+
|id |json_string                                                                             |parsed                                                         |
+---+----------------------------------------------------------------------------------------+---------------------------------------------------------------+
|1  |{"name": "John Doe", "age": 30}                                                         |{30, null, null, null, John Doe, null, null, null}             |
|2  |{"city": "New York", "country": "USA", "zipcode": "10001"}                              |{null, null, New York, USA, null, null, null, 10001}           |
|3  |{"product": "Laptop", "brand": "Dell", "specs": {"RAM": "16GB", "Storage": "512GB SSD"}}|{null, Dell, null, null, null, Laptop, {16GB, 512GB SSD}, null}|
+---+-----------------------------------------

# JSON from string

In [27]:
json_str = """[
    {
        "name":"Andre",
        "doc_list":[{"docid":"DOC001", "name":"bla001.txt"}, {"docid":"DOC002", "name":"bla002.txt"}]
    },
    {
        "name": "Noe",
        "doc_list":[{"docid":"DOC002", "name":"bla002.txt"}, {"docid":"DOC003", "name":"bla003.txt"}]
    }
]"""


In [28]:
json_str=re.sub(r"\n","",json_str)
sc = spark.sparkContext
schema = StructType([
    StructField("name", StringType(), True),
    StructField("doc_list", ArrayType(MapType(StringType(), StringType())), True)
])
df = spark.read.json(sc.parallelize([json_str]), schema)
df.show(truncate=False)

+-----+------------------------------------------------------------------------------+
|name |doc_list                                                                      |
+-----+------------------------------------------------------------------------------+
|Andre|[{docid -> DOC001, name -> bla001.txt}, {docid -> DOC002, name -> bla002.txt}]|
|Noe  |[{docid -> DOC002, name -> bla002.txt}, {docid -> DOC003, name -> bla003.txt}]|
+-----+------------------------------------------------------------------------------+



In [29]:
dfe = df.withColumn("item", F.explode("doc_list"))
dfe = dfe.withColumn("doc", dfe.item.getItem('name')).drop("item") \
        .groupBy('name').agg(F.collect_list("doc").alias("doclist")) \
        .withColumn('doc_names', F.concat_ws(',','doclist')).drop("doclist")
dfe.show(truncate=False)

+-----+---------------------+
|name |doc_names            |
+-----+---------------------+
|Noe  |bla002.txt,bla003.txt|
|Andre|bla001.txt,bla002.txt|
+-----+---------------------+

